# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 77kB/s 
     |████████████████████████████████| 204kB 49.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=dde95ae112f68b9255965e01dbb65f4d1beb2d4d91af6ce9c1bbb04cdc7645c5
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('lr').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression


In [5]:
df=spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [6]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
from pyspark.ml.feature import StringIndexer,VectorAssembler,VectorIndexer

In [9]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_new")

indexed=indexer.fit(df).transform(df)
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_new|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

In [16]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_new']

In [14]:
assembler=VectorAssembler(inputCols=[ 'Cruise_line_new',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew'],outputCol='features')

In [17]:
out=assembler.transform(indexed)
out.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_new|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|[16.0,6.0,30.2769...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|[16.0,6.0,30.2769...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|[1.0,26.0,47.262,...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|[1.0,11.0,110.0,2...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8

In [18]:
final_data=out.select(['crew','features'])

In [22]:
final_data.show()

+----+--------------------+
|crew|            features|
+----+--------------------+
|3.55|[16.0,6.0,30.2769...|
|3.55|[16.0,6.0,30.2769...|
| 6.7|[1.0,26.0,47.262,...|
|19.1|[1.0,11.0,110.0,2...|
|10.0|[1.0,17.0,101.353...|
| 9.2|[1.0,22.0,70.367,...|
| 9.2|[1.0,15.0,70.367,...|
| 9.2|[1.0,23.0,70.367,...|
| 9.2|[1.0,19.0,70.367,...|
|11.5|[1.0,6.0,110.2389...|
|11.6|[1.0,10.0,110.0,2...|
| 6.6|[1.0,28.0,46.052,...|
| 9.2|[1.0,18.0,70.367,...|
| 9.2|[1.0,17.0,70.367,...|
| 9.3|[1.0,11.0,86.0,21...|
|11.6|[1.0,8.0,110.0,29...|
|10.3|[1.0,9.0,88.5,21....|
| 9.2|[1.0,15.0,70.367,...|
| 9.3|[1.0,12.0,88.5,21...|
| 9.2|[1.0,20.0,70.367,...|
+----+--------------------+
only showing top 20 rows



In [23]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [26]:
lr=LinearRegression(labelCol='crew')
lr_model=lr.fit(train_data)

In [27]:
pred=lr_model.evaluate(test_data)

In [28]:
pred.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-2.10942374678779...|
|-1.88737914186276...|
|4.218847493575595...|
|4.440892098500626...|
|4.440892098500626...|
|5.773159728050814...|
|-8.88178419700125...|
|-8.88178419700125...|
|-6.21724893790087...|
|8.881784197001252...|
|2.664535259100375...|
|-8.88178419700125...|
|7.105427357601002...|
|7.105427357601002...|
|9.769962616701378...|
|-2.66453525910037...|
|-4.44089209850062...|
|1.776356839400250...|
|2.664535259100375...|
|                 0.0|
+--------------------+
only showing top 20 rows



In [29]:
pred.r2

1.0

In [30]:
pred.rootMeanSquaredError

3.973088899775122e-15

The model performs so well as the dataset is artificially created and is nearly perfect for this task